In [1]:
# %%
import numpy as np
from numpy.linalg import det, inv, norm
from numpy.random import normal
from scipy.optimize import minimize, Bounds, least_squares
import matplotlib.pyplot as plt
from matplotlib import cm
import math
import random
from functools import cache
# %%
np.random.seed(23)
np.random.RandomState = 22
random.seed(22)
np.set_printoptions(precision=5)
plt.rcParams['figure.figsize'] = [10, 4]
# # 1. Исходные данные

In [2]:
def F_(th_1):
    return np.matrix([[th_1, 0.55], [-0.1, 0.5]])

def PSI_(th_2):
    return np.matrix([1, th_2]).T

G = np.matrix([1, 1]).T
H = np.matrix([1, 0])
Q = np.matrix(0.1)
R = np.matrix(0.05)
x0 = np.matrix([0, 0]).T
P0 = np.matrix([[0.1, 0], [0, 0.1]])
I = np.eye(2)
N = 30 # Размерность сигнала U
s = 2 # Размерность th
th_true = np.array([-0.8, 1])
th_1_range = [-2, -0.05]
th_2_range = [0.01, 1.5]
U = np.matrix(np.ones(N+1)*5).T

In [3]:
# # 2. Отклик
# %%
def make_Y(th, err=1, err2=1):
    X = np.matrix(np.zeros((2, N+1)))
    Y = np.matrix(np.zeros((N+1, 1)))
    F = F_(th[0])
    PSI = PSI_(th[1])
    X[:, 0] = x0
    Y[0] = H@X[:, 0] + normal(-np.sqrt(R)/2, np.sqrt(R))*err
    for k in range(1, N+1):
        wi = normal(-np.sqrt(R)/2, np.sqrt(Q), (1,2))*err
        vi = normal(-np.sqrt(Q)/2, np.sqrt(R))*err*err2
        X[:, k] = F@X[:, k-1] + PSI@U[k] + (G@wi)[0].T
        Y[k] = H@X[:, k] + vi
    return Y, X

In [4]:
def HI(th, Y):
    hi = N*np.log(2*np.pi)
    P_k_k = P0
    x_k_k = x0
    F = F_(th[0])
    PSI = PSI_(th[1])
    for k in range(N):
        P_k1_k = F@P_k_k@F.T + G@Q@G.T
        B_k1 = H@P_k1_k@H.T + R
        K_k1 = P_k1_k@H.T@inv(B_k1)
        P_k1_k1 = (I - K_k1@H)@P_k1_k
        x_k1_k = F@x_k_k + PSI@U[k]
        e_k1 = Y[k+1] - H@x_k1_k
        x_k_k = x_k1_k + K_k1@e_k1
        delta = e_k1.T@inv(B_k1)@e_k1
        hi += math.log(det(B_k1)) + delta
        P_k_k = P_k1_k1
    return np.float64(hi/2)

print(HI(np.array([-1.2, 1.81]), make_Y(np.array([-1.9, 0.81]), 0)[0]))
print(HI(th_true, make_Y(th_true, 0)[0]))

5.6450990306036344e+16
0.05376917009959059


C:\Users\Mark\AppData\Local\Temp\ipykernel_22128\2425434409.py:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return np.float64(hi/2)


In [5]:
def gradHI(th, Y):
    dF = [np.matrix([[1,0],[0,0]]), np.matrix(np.zeros((2,2)))] 
    dPSI = [np.matrix(np.zeros((2,1))), np.matrix([0,1]).T] 
    dx0 = np.array([[[0],[0]],[[0],[0]]]) # dG, dH, dQ, dR, dx0, dP0 - нулевые матрицы 
    
    dHi = np.zeros(len(th)) 
    
    F = F_(th[0]) 
    PSI = PSI_(th[1]) 
    
    P_k_k = P0 
    dP_k_k = [np.matrix(np.zeros((2,2))), np.matrix(np.zeros((2,2)))] 
    
    x_k_k = x0 
    dx_k_k = dx0 
    
    for k in range(N): 
        P_k1_k = F@P_k_k@F.T + G@Q@G.T 
        B_k1 = H@P_k1_k@H.T + R 
        K_k1 = P_k1_k@H.T@inv(B_k1) 
        P_k1_k1 = (I - K_k1@H)@P_k1_k 
        
        dP_k1_k = [None, None] 
        dB_k1 = [None, None] 
        dK_k1 = [None, None] 
        dx_k1_k1 = [None, None] 
        
        for i in range(s): 
            dP_k1_k[i] = dF[i]@P_k_k@F.T + F@dP_k_k[i]@F.T + F@P_k_k@dF[i].T 
            dB_k1[i] = H@dP_k1_k[i]@H.T
            dK_k1[i] = (dP_k1_k[i]@H.T - P_k1_k@H.T@inv(B_k1)@dB_k1[i])@inv(B_k1) 
            dP_k_k[i] = (I - K_k1@H)@dP_k1_k[i] - dK_k1[i]@H@P_k1_k 
            
            delta = np.zeros((s, 1)) 
            
            x_k1_k = F@x_k_k + PSI@U[k] 
            e_k1 = Y[k+1] - H@x_k1_k 
            x_k1_k1 = x_k1_k + K_k1@e_k1 
            
            for i in range(s): 
                dx_k1_k = dF[i]@x_k_k + F@dx_k_k[i] + dPSI[i]@U[k] 
                de_k1 = -H@dx_k1_k 
                dx_k1_k1[i] = dx_k1_k + dK_k1[i]@e_k1 + K_k1@de_k1
                delta[i] += float( 
                    de_k1.T@inv(B_k1)@e_k1 -\
                    1/2 * e_k1.T@inv(B_k1)@dB_k1[i]@inv(B_k1)@e_k1
                )
                dHi[i] += 1/2 * np.trace(inv(B_k1)@dB_k1[i]) + delta[i]
                
            P_k_k = P_k1_k1 
            x_k_k = x_k1_k1 
            dx_k_k = dx_k1_k1
            
        return dHi

a = gradHI(np.array([-1.9, 1.81]), make_Y(np.array([-1.9, 0.81]), 0)[0]) 
print(type(a), a)

C:\Users\Mark\AppData\Local\Temp\ipykernel_22128\3686827930.py:44: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  delta[i] += float(
C:\Users\Mark\AppData\Local\Temp\ipykernel_22128\3686827930.py:48: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  dHi[i] += 1/2 * np.trace(inv(B_k1)@dB_k1[i]) + delta[i]


ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)